In [1]:
!pip install llama-index transformers accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 47.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 85.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.6 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensio

In [2]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.0 MB/s eta 0:00:00


In [3]:
import torch
from transformers import BitsAndBytesConfig
from llama_index.prompts import PromptTemplate
from llama_index.llms import HuggingFaceLLM

In [4]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

def messages_to_prompt(messages):
  prompt = ""
  for message in messages:
    if message.role == 'system':
      prompt += f"<|system|>\n{message.content}</s>\n"
    elif message.role == 'user':
      prompt += f"<|user|>\n{message.content}</s>\n"
    elif message.role == 'assistant':
      prompt += f"<|assistant|>\n{message.content}</s>\n"

  # ensure we start with a system prompt, insert blank if needed
  if not prompt.startswith("<|system|>\n"):
    prompt = "<|system|>\n</s>\n" + prompt

  # add final assistant prompt
  prompt = prompt + "<|assistant|>\n"

  return prompt


In [5]:
llm = HuggingFaceLLM(
    model_name="HuggingFaceH4/zephyr-7b-alpha",
    tokenizer_name="HuggingFaceH4/zephyr-7b-alpha",
    query_wrapper_prompt=PromptTemplate("<|system|>\n</s>\n<|user|>\n{query_str}</s>\n<|assistant|>\n"),
    context_window=3900,
    max_new_tokens=2000,
    model_kwargs={"quantization_config": quantization_config},
    # tokenizer_kwargs={},
    generate_kwargs={"temperature": 0.7, "top_k": 50, "top_p": 0.95},
    messages_to_prompt=messages_to_prompt,
    device_map="auto",
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/628 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [6]:
from llama_index import ServiceContext, set_global_service_context

service_context = ServiceContext.from_defaults(llm=llm, embed_model="local:BAAI/bge-small-en-v1.5")

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [7]:
set_global_service_context(service_context)

In [8]:
from llama_index import SimpleDirectoryReader, VectorStoreIndex
from llama_index.readers.file.base import (
    DEFAULT_FILE_READER_CLS,
    ImageReader,
)
from llama_index.response.notebook_utils import (
    display_response,
    display_image,
)
from llama_index.indices.query.query_transform.base import (
    ImageOutputQueryTransform,
)

In [9]:
filename_fn = lambda filename: {"file_name": filename}


In [10]:
llama_reader = SimpleDirectoryReader(
    input_dir="/content/drive/MyDrive/llama",
    file_metadata=filename_fn,
)
llama_documents = llama_reader.load_data()

In [11]:
llama_index = VectorStoreIndex.from_documents(llama_documents)

In [14]:
from llama_index.query_engine import TransformQueryEngine


query_engine = llama_index.as_query_engine(similarity_top_k=2)
query_engine = TransformQueryEngine(
    query_engine, query_transform=ImageOutputQueryTransform(width=400)
)
llama_response = query_engine.query(
    "Show an image of a store bill and explain briefly",
)

In [15]:
display_response(llama_response)


**`Final Response:`** To show an image of a store bill, you can use an HTML <img> tag with the source of the image and a width of 400 pixels. Here's an example:

```html
<img src="data/store_bill.jpg" width="400">
```

In this example, "data/store_bill.jpg" is the path to the image file, and "400" is the width of the image in pixels. You can replace the path and width values with your own.

In [16]:
from llama_index.readers.file.base import DEFAULT_FILE_READER_CLS
from llama_index.readers.file.image_reader import ImageReader

image_parser =ImageReader(
    keep_image=True,
    parse_text=True
    )
file_extractor = DEFAULT_FILE_READER_CLS
file_extractor.update({
    ".jpg": image_parser,
    ".png": image_parser,
    ".jpeg": image_parser,
    })

preprocessor_config.json:   0%|          | 0.00/362 [00:00<?, ?B/s]

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


tokenizer_config.json:   0%|          | 0.00/536 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/1.30M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.02M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/335 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.74k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/806M [00:00<?, ?B/s]

In [17]:
receipt_reader = SimpleDirectoryReader(
    input_dir="/content/drive/MyDrive/Invoices",
    file_metadata=filename_fn,
    file_extractor=file_extractor,
)
receipt_documents = receipt_reader.load_data()
print(len(receipt_documents))

16


In [18]:
receipts_index = VectorStoreIndex.from_documents(receipt_documents)

In [32]:
from llama_index.query_engine import TransformQueryEngine
query_engine = receipts_index.as_query_engine()

receipts_response = query_engine.query(
    "Retrieve details from my store bills, including Bill Date, Bill No., Bill Products, Quantity, Price, etc.",
)


In [33]:
display_response(receipts_response)

**`Final Response:`** To retrieve details from your store bills, including Bill Date, Bill No., Bill Products, Quantity, Price, etc., you can refer to the provided context information. The details are available in the given text, which includes the following:

- Bill Date: 3-8-28/455
- Bill No.: 501801013-00205
- Bill Products:
  - TATA SALT-lkg
  - CGST @ 2.50%
  - SGST @ 2.50%
  - 110100 서울특별시 WHEA-Skg
  - CGST @ 9.00% SGS
  - 330510
  - TREAME MHO SBO
  - INDICA NAT B
  - RIN LIQUID DES Samil
  - COLLN GLASSE TEE-Jit
  - DOMEX D'SNSACT-11t
  - REFLECT STAR LESS S
  - FIGA ACEITE-500ml

- Quantity:
  - 1
  - 1
  - 3
  - 3
  - 1
  - 2
  - 3
  - 1
  - 5

- Price:
  - 2500
  - 25.00
  - 75.00
  - 258.00
  - 399.00
  - 183.00
  - 370.00
  - 30.00
  - 499.00
  - 499.00
  - 17.00
  - 238.10
  - 2500.74
  - 85.03

In addition, the new context provides GST Breakup Details for the following products:

- GST Breakup Details:
  - GST: 75.00
  - AND Amount: 10.00
  - Total: 85.03

Note that the provided context information may contain additional details that are not listed here.

Refined Answer:

To retrieve details from your store bills, including Bill Date, Bill No., Bill Products, Quantity, Price, etc., you can refer to the provided context information. The details are available in the given text, which includes the following:

- Bill Date: 3-8-28/455
- Bill No.: 501801013-00205
- Bill Products:
  - TATA SALT-lkg
  - CGST @ 2.50%
  - SGST @ 2.50%
  - 110100 서울특별시 WHEA-Skg
  - CGST @ 9.00% SGS
  - 330510
  - TREAME MHO SBO
  - INDICA NAT B
  - RIN LIQUID DES Samil
  - COLLN GLASSE TEE-Jit
  - DOMEX D'SNSACT-11t
  - REFLECT STAR LESS S
  - FIGA ACEITE-500ml

- Quantity:
  - 1
  - 1
  - 3
  - 3
  - 1
  - 2
  - 3
  - 1
  - 5

- Price:
  - 2500
  - 25.00
  - 75.00
  - 258.00
  - 399.00
  - 183.00
  - 370.00
  - 30.00
  - 499.00
  - 499.00
  - 17.00
  - 238.10
  - 2500.74
  - 85.03

In addition, the new context provides GST Breakup Details for the following products:

- GST Breakup Details:
  - GST: 75.00
  - AND Amount: 10.00
  - Total: 85.03

Note that the provided context information may contain additional details that are not listed here.

Refined Answer:

To retrieve details from your store bills, including Bill Date, Bill No., Bill Products, Quantity, Price, etc., you can refer to the provided context information. The details are available in the given text, which includes the following:

- Bill Date: 3-8-28/455
- Bill No.: 501801013-00205
- Bill Products:
  - TATA SALT-lkg
  - CGST @ 2.50%
  - SGST @ 2.50%
  - 110100 서울특별시 WHEA-Skg
  - CGST @ 9.00% SGS
  - 330510
  - TREAME MHO SBO
  - INDICA NAT B
  - RIN LIQUID DES Samil
  - COLLN GLASSE TEE-Jit
  - DOMEX D'SNSACT-11t
  - REFLECT STAR LESS S
  - FIGA ACEITE-500ml

- Quantity:
  - 1
  - 1
  - 3
  - 3
  - 1
  - 2
  - 3
  - 1
  - 5

- Price:
  - 2500
  - 25.00
  - 75.00
  - 258.00
  - 399.00
  - 183.00
  - 370.00
  - 30.00
  - 499.00
  - 499.00
  - 17.00
  - 238.10
  - 2500.74
  - 85.03

In addition, the new context provides GST Breakup Details for the following products:

- GST Breakup Details:
  - GST: 75.00
  - AND Amount: 10.00
  - Total: 85.03

Note that the provided context information may contain additional details that are not listed here.

Refined Answer:

To retrieve details from your store bills, including Bill Date, Bill No., Bill Products, Quantity, Price, etc., you can refer to the provided context information. The details are available in the given text, which includes the following:

- Bill Date: 3-8-28/455
- Bill No.: 501801013-00205
- Bill Products:
  - TATA SALT-lkg
  - CGST @ 2.50%
  - SGST @ 2.50%
  - 110100 서울특별시 WHEA-Skg
  - CGST @ 9.00% SGS
  - 330510
  - TREAME MHO SBO
  - INDICA NAT B
  - RIN LIQUID DES Samil
  - COLLN GLASSE TEE-Jit
  - DOMEX D'SNSACT-11t
  - REFLECT STAR LESS S
  - FIGA ACEITE-500ml

- Quantity:
  - 1
  - 1
  - 3
  - 3
  - 1
  - 2
  - 3
  - 1
  - 5

- Price:
  - 2500
  - 25.00
  - 75.00
  - 258.00
  - 399.00
  - 183.00
  - 370.00
  - 30